In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-70b-versatile",
    groq_api_key="your api key here"
)
response=llm.invoke("Who landed on moon first?")
print(response.content)

Neil Armstrong was the first person to land on the moon.  He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader("https://jobs.nike.com/job/R-37192")

page_data= loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Senior Solutions Data Engineer, GPS

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Solutions Data Engineer, GPSBeaverton, OregonBecome a Part of the NIKE, In

In [3]:
from langchain_core.prompts import PromptTemplate

# Create the prompt template
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: 'job'.'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Chain the template with the language model (llm)
chain_extract = prompt_extract | llm

# Invoke the chain to extract information
res= chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```
{
  "job": {
    "role": "Senior Solutions Data Engineer, GPS",
    "experience": "5+ years of experience working as an application engineer focused on data product engineering",
    "skills": [
      "SQL",
      "Java",
      "Python",
      "PySpark",
      "Analytics technologies such as Snowflake and Databricks",
      "Cloud technologies such as AWS, Azure, GCP",
      "DevOps practices",
      "Agile methodologies",
      "Project management"
    ],
    "description": "We are looking for a Senior Solutions Data Engineer to join the GPS Solutions team to drive and deploy governed technology solutions to empower GPS Capabilities. These solutions will focus on the design, development and management of data products and data sets."
  }
}
```


In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Solutions Data Engineer, GPS',
 'experience': '5+ years of experience working as an application engineer focused on data product engineering',
 'skills': ['SQL',
  'Java',
  'Python',
  'PySpark',
  'Analytics technologies such as Snowflake and Databricks',
  'Cloud technologies such as AWS, Azure, GCP',
  'DevOps practices',
  'Agile methodologies',
  'Project management'],
 'description': 'We are looking for a Senior Solutions Data Engineer to join the GPS Solutions team to drive and deploy governed technology solutions to empower GPS Capabilities. These solutions will focus on the design, development and management of data products and data sets. You will be responsible for technical engineering delivery of location related data that will drive meaningful and actionable insights tied to optimizing NIKE’s global physical real estate portfolio.'}

In [10]:
type(json_res)

dict

In [5]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
print(df)

                               Techstack  \
0                React, Node.js, MongoDB   
1               Angular,.NET, SQL Server   
2      Vue.js, Ruby on Rails, PostgreSQL   
3                  Python, Django, MySQL   
4              Java, Spring Boot, Oracle   
5             Flutter, Firebase, GraphQL   
6                  WordPress, PHP, MySQL   
7                    Magento, PHP, MySQL   
8         React Native, Node.js, MongoDB   
9                  iOS, Swift, Core Data   
10       Android, Java, Room Persistence   
11             Kotlin, Android, Firebase   
12       Android TV, Kotlin, Android NDK   
13                     iOS, Swift, ARKit   
14        Cross-platform, Xamarin, Azure   
15          Backend, Kotlin, Spring Boot   
16         Frontend, TypeScript, Angular   
17    Full-stack, JavaScript, Express.js   
18  Machine Learning, Python, TensorFlow   
19               DevOps, Jenkins, Docker   

                                                Links  
0                 h

In [6]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=["Expertise in React Native","Experience in Python"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [21]:
job= json_res
job['skills']

['SQL',
 'Java',
 'Python',
 'PySpark',
 'Analytics technologies such as Snowflake and Databricks',
 'Cloud technologies such as AWS, Azure, GCP',
 'DevOps practices',
 'Agile methodologies',
 'Project management']

In [25]:
prompt_email = PromptTemplate.from_template(
"""
## JOB DESCRIPTION:
{job_description}

## INSTRUCTION:
You are Faisal Riaz, a business development executive at Celystik Labs. Celystik Labs is a research and development studio specializing in Blockchain, AI,Data Engineering and decentralized finance. Our core innovations include Intelligent Liquidity Provisioning Agents and TokenGPT, an AI-driven assistant for blockchain research and development. We are dedicated to optimizing liquidity strategies in DeFi platforms and assisting with token engineering using cutting-edge AI and Reinforcement Learning methodologies.

Your job is to write a cold email to the client regarding the job mentioned above, describing how Celystik Labs can fulfill their needs.As celystik have data engineers with expertise in this field as mentioned in job description.

*🟡 Add the most relevant ones from the following links to showcase Celystik's solutions. 
Remember you are Faisal Riaz, BDE at Celystik Labs. 
Do not provide a preamble.*

#### EMAIL (NO PREAMBLE):
"""
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Data Engineering Solutions for NIKE's GPS Capabilities

Dear Hiring Manager,

I came across the Senior Solutions Data Engineer role at NIKE and was impressed by the company's commitment to driving innovation through data-driven solutions. As a Business Development Executive at Celystik Labs, I'd like to introduce our team of expert data engineers who can help fulfill your requirements.

Our data engineers possess 5+ years of experience in application engineering, focusing on data product engineering, and have expertise in the skills you've mentioned, including SQL, Java, Python, PySpark, Analytics technologies (Snowflake and Databricks), Cloud technologies (AWS, Azure, GCP), DevOps practices, Agile methodologies, and Project management.

At Celystik Labs, we've developed cutting-edge solutions that can complement your GPS Capabilities. Our Intelligent Liquidity Provisioning Agents can be adapted to optimize data product engineering, ensuring seamless data management and